## 資料清理

In [26]:
import pandas as pd

#讀取資料
dataset = pd.read_csv('LoanStats.csv')

#移除空白欄位
dataset = dataset.iloc[:,2:111]         
empty_cols = [i for i in range(45,72)]   
dataset = dataset.drop(dataset.columns[empty_cols],axis=1)
data_with_loanstatus_sliced = dataset[(dataset['loan_status']=="Fully Paid") | (dataset['loan_status']=="Charged Off")]

#轉換目標編碼
di = {"Fully Paid":0, "Charged Off":1}   
Dataset_withBoolTarget= data_with_loanstatus_sliced.replace({"loan_status": di})

/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0,19,47,123,124,125,128,129,130,133) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
Dataset_withBoolTarget['loan_status'].value_counts()
print("Current shape of dataset :",Dataset_withBoolTarget.shape)
Dataset_withBoolTarget.head(3)

Current shape of dataset : (411273, 82)


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,12000.0,12000.0,12000.0,36 months,7.62%,373.94,A,A3,Systems Engineer,3 years,...,0.0,3.0,100.0,100.0,0.0,0.0,233004.0,46738.0,14800.0,53404.0
1,27050.0,27050.0,27050.0,36 months,10.99%,885.46,B,B2,Team Leadern Customer Ops & Systems,10+ years,...,0.0,1.0,100.0,25.0,0.0,0.0,138554.0,70186.0,35700.0,33054.0
2,12000.0,12000.0,12000.0,36 months,11.99%,398.52,B,B3,LTC,10+ years,...,0.0,3.0,100.0,1.0,0.0,0.0,365874.0,44327.0,10700.0,57674.0


In [4]:
#移除空白列
dataset=Dataset_withBoolTarget.dropna(thresh = 340000,axis=1) 
print("Current shape of dataset :",dataset.shape)

Current shape of dataset : (411273, 76)


In [5]:
#移除欄位
del_col_names = ["delinq_2yrs",  "last_pymnt_d", "chargeoff_within_12_mths","delinq_amnt","emp_title", "term", "emp_title", "pymnt_plan","purpose","title", "zip_code", "verification_status", "dti","earliest_cr_line", "initial_list_status", "out_prncp",
"pymnt_plan", "num_tl_90g_dpd_24m", "num_tl_30dpd", "num_tl_120dpd_2m", "num_accts_ever_120_pd", "delinq_amnt", 
"chargeoff_within_12_mths", "total_rec_late_fee", "out_prncp_inv", "issue_d"] 
dataset = dataset.drop(labels = del_col_names, axis = 1) 
print("Current shape of dataset :",dataset.shape)

Current shape of dataset : (411273, 54)


In [10]:
#篩選欄位
features = ['funded_amnt','emp_length','annual_inc','home_ownership','grade',
            "last_pymnt_amnt", "mort_acc", "pub_rec", "int_rate", "open_acc","num_actv_rev_tl",
            "mo_sin_rcnt_rev_tl_op","mo_sin_old_rev_tl_op","bc_util","bc_open_to_buy",
            "avg_cur_bal","acc_open_past_24mths",'loan_status'] #'sub_grade' #selecting final features #'addr_state''tax_liens',
Final_data = dataset[features] #19 features with target var
Final_data["int_rate"] = Final_data["int_rate"].apply(lambda x:float(x[:-1]) ) #reomving % sign, conv to float  - int_rate column
Final_data= Final_data.reset_index(drop=True)
print("Current shape of dataset :",Final_data.shape)

Current shape of dataset : (411273, 18)


/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


## 資料轉換

In [12]:
#Data encoding
Final_data['grade'] = Final_data['grade'].map({'A':7,'B':6,'C':5,'D':4,'E':3,'F':2,'G':1})
Final_data["home_ownership"] = Final_data["home_ownership"].map({"MORTGAGE":6,"RENT":5,"OWN":4,"OTHER":3,"NONE":2,"ANY":1})
Final_data["emp_length"] = Final_data["emp_length"].fillna('0')
Final_data["emp_length"] = Final_data["emp_length"].replace({'years':'','year':'',' ':'','<':'','\+':'','n/a':'0'}, regex = True)
Final_data["emp_length"] = Final_data["emp_length"].apply(lambda x:int(x))
print("Current shape of dataset :",Final_data.shape)
Final_data.head()

Current shape of dataset : (411273, 18)


,funded_amnt,emp_length,annual_inc,home_ownership,grade,last_pymnt_amnt,mort_acc,pub_rec,int_rate,open_acc,num_actv_rev_tl,mo_sin_rcnt_rev_tl_op,mo_sin_old_rev_tl_op,bc_util,bc_open_to_buy,avg_cur_bal,acc_open_past_24mths,loan_status
0,12000.0,3,96500.0,6,7,2927.22,1.0,0.0,7.62,17.0,5.0,10.0,118.0,83.5,2441.0,11783.0,4.0,0
1,27050.0,10,55000.0,4,6,6074.19,4.0,0.0,10.99,14.0,4.0,16.0,326.0,53.9,16473.0,9570.0,3.0,0
2,12000.0,10,130000.0,6,6,398.28,3.0,0.0,11.99,9.0,5.0,4.0,193.0,93.0,3567.0,36362.0,4.0,0
3,28000.0,5,325000.0,6,7,23915.86,5.0,0.0,7.62,15.0,5.0,5.0,229.0,67.1,13901.0,53306.0,6.0,0
4,12000.0,4,60000.0,5,6,3775.55,0.0,0.0,10.99,15.0,7.0,1.0,48.0,15.9,15216.0,476.0,8.0,0


## 使用平均值填補遺失值

In [14]:
Final_data.fillna(Final_data.mean(),inplace = True)
print("Current shape of dataset :",Final_data.shape)

Current shape of dataset : (411273, 18)


## 將特徵值標準化

In [16]:
from sklearn import preprocessing,metrics 
scl = preprocessing.StandardScaler() 
fields = Final_data.columns.values[:-1]
data_clean = pd.DataFrame(scl.fit_transform(Final_data[fields]), columns = fields)
data_clean['loan_status'] = Final_data['loan_status']
data_clean['loan_status'].value_counts()

0    340444
1     70829
Name: loan_status, dtype: int64

## 合併清理過後的資料

In [17]:
loanstatus_0 = data_clean[data_clean["loan_status"]==0]
loanstatus_1 = data_clean[data_clean["loan_status"]==1]
subset_of_loanstatus_0 = loanstatus_0.sample(n=5500)
subset_of_loanstatus_1 = loanstatus_1.sample(n=5500)
data_clean = pd.concat([subset_of_loanstatus_1, subset_of_loanstatus_0])
data_clean = data_clean.sample(frac=1).reset_index(drop=True)
print("Current shape of dataset :",data_clean.shape)
data_clean.head()

Current shape of dataset : (11000, 18)


,funded_amnt,emp_length,annual_inc,home_ownership,grade,last_pymnt_amnt,mort_acc,pub_rec,int_rate,open_acc,num_actv_rev_tl,mo_sin_rcnt_rev_tl_op,mo_sin_old_rev_tl_op,bc_util,bc_open_to_buy,avg_cur_bal,acc_open_past_24mths,loan_status
0,1.634916,-0.239455,0.587600,0.890293,-0.146201,3.799844,1.470269,-0.323443,0.008930,1.134412,0.091852,-0.220361,1.466732,0.740471,-0.304638,1.011872,-0.075437,0
1,-0.299476,0.308420,-0.214289,0.890293,-0.146201,-0.665392,0.079569,-0.323443,-0.216606,-0.269915,-0.248869,1.394208,3.697718,1.102137,-0.610760,0.400891,-1.160718,0
2,-0.480826,-0.787331,-0.811044,-0.640718,-0.146201,-0.435546,-0.847563,-0.323443,0.758436,-1.072387,-0.589591,-0.543275,-0.377247,0.405737,-0.420588,-0.662427,-0.075437,0
3,2.481212,1.130234,1.426787,0.890293,-1.679450,4.936103,0.543136,-0.323443,1.719808,0.331940,0.091852,-0.478692,3.242415,-1.244849,0.723499,0.113804,1.009844,0
4,-0.650085,0.582358,-0.624558,-0.640718,0.620423,-0.630130,-0.847563,-0.323443,-0.444419,-0.269915,-0.248869,-0.091196,-0.126830,1.036730,-0.502744,-0.299160,-0.798958,1


In [18]:
data_clean.corr()

,funded_amnt,emp_length,annual_inc,home_ownership,grade,last_pymnt_amnt,mort_acc,pub_rec,int_rate,open_acc,num_actv_rev_tl,mo_sin_rcnt_rev_tl_op,mo_sin_old_rev_tl_op,bc_util,bc_open_to_buy,avg_cur_bal,acc_open_past_24mths,loan_status
funded_amnt,1.000000,0.130238,0.237591,0.190555,-0.201386,0.312273,0.246359,-0.083430,0.198911,0.200747,0.160271,0.054173,0.187949,0.049834,0.178899,0.230468,0.001896,0.083734
emp_length,0.130238,1.000000,0.069117,0.135227,-0.019348,0.050546,0.163817,0.000115,0.022089,0.032102,0.098129,0.020974,0.102765,0.031267,0.019824,0.094393,0.012368,-0.000895
annual_inc,0.237591,0.069117,1.000000,0.110078,0.012339,0.107694,0.164528,0.000667,-0.018964,0.123040,0.060315,0.026771,0.107527,-0.009095,0.105159,0.223173,0.036640,-0.036264
home_ownership,0.190555,0.135227,0.110078,1.000000,0.054383,0.102596,0.412157,-0.000044,-0.054910,0.103925,0.034711,0.020748,0.101327,0.017305,0.052392,0.362853,0.038565,-0.046882
grade,-0.201386,-0.019348,0.012339,0.054383,1.000000,0.004058,0.051791,-0.053545,-0.961970,-0.029418,-0.110573,0.106194,0.102451,-0.250925,0.260021,0.078208,-0.176556,-0.300218
last_pymnt_amnt,0.312273,0.050546,0.107694,0.102596,0.004058,1.000000,0.145807,-0.023145,-0.007635,0.055449,0.008620,0.037221,0.063748,-0.047247,0.123446,0.158288,0.012494,-0.470560
mort_acc,0.246359,0.163817,0.164528,0.412157,0.051791,0.145807,1.000000,0.016429,-0.054463,0.112534,0.032376,0.035851,0.277259,-0.018422,0.129064,0.443934,0.050730,-0.057100
pub_rec,-0.083430,0.000115,0.000667,-0.000044,-0.053545,-0.023145,0.016429,1.000000,0.046647,-0.045862,-0.031190,-0.043641,0.075473,-0.056985,-0.079008,-0.041945,0.037490,0.015804
int_rate,0.198911,0.022089,-0.018964,-0.054910,-0.961970,-0.007635,-0.054463,0.046647,1.000000,0.019964,0.113842,-0.106430,-0.109017,0.279156,-0.282128,-0.080607,0.168382,0.302242
open_acc,0.200747,0.032102,0.123040,0.103925,-0.029418,0.055449,0.112534,-0.045862,0.019964,1.000000,0.638986,-0.212445,0.129331,-0.101595,0.259835,-0.089236,0.458223,0.030297


## 將資料區分為訓練與測試資料集

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_clean.iloc[:,:-1], data_clean.iloc[:,-1], test_size=0.2, random_state=42)
bs_train, bs_test = train_test_split(data_clean, test_size = 0.2, random_state=42) #just for boots

## Recursive Feature Elimination

In [23]:
from sklearn import linear_model,svm
from sklearn.feature_selection import RFE
# create the RFE model and select 3 attributes
clf_LR = linear_model.LogisticRegression(C=1e30)
clf_LR.fit(X_train,y_train)
rfe = RFE(clf_LR, 10)
rfe = rfe.fit(data_clean.iloc[:,:-1].values, data_clean.iloc[:,-1].values)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[ True False  True False  True  True False False  True  True False False
 False  True  True  True  True]
[1 6 1 5 1 1 7 4 1 1 3 8 2 1 1 1 1]


## PCA

In [24]:
from sklearn.decomposition import PCA 
pca = PCA(n_components=10, whiten=True)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
explained_variance = pca.explained_variance_ratio_
print('Expected Variance is '+ str(explained_variance))

Expected Variance is [0.184648   0.13960312 0.11890045 0.0991704  0.06341064 0.06028455
 0.05645515 0.04934004 0.04127878 0.03849465]


## 資料篩選

In [25]:
features = ['funded_amnt','annual_inc','grade',"last_pymnt_amnt", "int_rate",
            "mo_sin_rcnt_rev_tl_op","mo_sin_old_rev_tl_op","bc_util","bc_open_to_buy","acc_open_past_24mths","loan_status"]
X_train, X_test = X_train[features[:-1]], X_test[features[:-1]]
data_clean = data_clean[features]
print(X_train.shape)
print(data_clean.shape)

(8800, 10)
(11000, 11)
